# Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
import xgboost as xgb
import lightgbm as lgb

from datetime import datetime

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler


from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor, StackingClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn import svm

from catboost import CatBoostClassifier
from xgboost import XGBClassifier

# Data Loading

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
train

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience,blueWins
0,0,0,5,8,6,0,0,14536,17256,0
1,1,1,10,1,5,0,0,14536,17863,0
2,2,0,3,10,2,0,0,17409,17256,0
3,3,1,7,10,8,0,0,19558,18201,0
4,4,0,4,9,4,0,0,17409,17256,0
...,...,...,...,...,...,...,...,...,...,...
7995,9993,1,9,6,14,0,0,18513,18201,1
7996,9994,0,3,10,4,0,0,17381,19797,0
7997,9995,1,9,3,9,2,1,18274,18491,1
7998,9997,0,5,10,6,1,1,18274,18491,1


In [4]:
test

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience
0,9,0,7,6,6,0,0,16961,18201
1,15,0,6,6,6,2,1,18513,18021
2,18,1,6,4,3,0,0,13475,17256
3,23,0,5,4,7,0,0,17409,17256
4,31,0,10,8,9,0,0,18117,18472
...,...,...,...,...,...,...,...,...,...
1995,9971,0,6,3,7,0,0,17409,17256
1996,9980,0,4,4,4,0,0,17409,17256
1997,9983,1,6,3,3,1,1,18513,18201
1998,9996,1,10,9,9,1,1,18513,18201


# Feature

- 各特長量の説明

| ヘッダ名称            | データ型 | 説明                                                       |
|---------------------|-------|----------------------------------------------------------|
| gameId             | int   | ゲームID                                                    |
| blueFirstBlood      | int   | ゲームの最初のキル。青チームが最初のキルを行った場合は1、それ以外の場合は0 |
| blueKills           | int   | 青チームによって殺された敵の数                                     |
| blueDeaths          | int   | 青チームの死亡者数                                            |
| blueAssists         | int   | 青チームのキルアシストの数                                      |
| blueEliteMonsters   | int   | 青チームによって殺されたエリートモンスターの数（ドラゴンとヘラルド）       |
| blueDragons         | int   | 青チームによって殺されたドラゴンの数                               |
| blueTotalGold       | int   | 青チームの得たゴールド合計                                       |
| blueTotalExperience | int   | 青チームの得た経験値合計                                       |
| blueWins            | int   | 目的変数（青チームが勝った場合は1、それ以外の場合は0。）                |


# Feature Engineering

- Add new feature

In [5]:
def create_features(df):
    """
    与えられたデータフレームに対して特徴量エンジニアリングを行う関数。

    Args:
    df (DataFrame): 特徴量エンジニアリングを行うデータフレーム

    Returns:
    DataFrame: 新しい特徴量が追加されたデータフレーム
    """
    # キルとデスの比率
    df['killDeathRatio'] = df['blueKills'] / (df['blueDeaths'] + 1)  # ゼロ除算を防ぐために+1

    # キルとデスの差
    df['killDeathDiff'] = df['blueKills'] - df['blueDeaths']
    # 負の値を0で置き換え
    df['killDeathDiff'] = df['killDeathDiff'].clip(lower=0)

    # アシストの割合
    df['assistRate'] = df['blueAssists'] / (df['blueKills'] + 1)

    # ヘラルドの討伐数
    df['blueHerald'] = df['blueEliteMonsters'] - df['blueDragons']
    # blueHerald列の負の値を0で置き換え
    df['blueHerald'] = df['blueHerald'].clip(lower=0)

    # エリートモンスターとドラゴンの比率
    df['eliteMonsterDragonRatio'] = df['blueEliteMonsters'] / (df['blueDragons'] + 1)
    # エリートモンスターとヘラルドの比率
    df['eliteMonsterHeraldRatio'] = df['blueEliteMonsters'] / (df['blueHerald'] + 1)

    # ゴールドと経験値の対数変換
    df['logTotalGold'] = np.log1p(df['blueTotalGold'])
    df['logTotalExperience'] = np.log1p(df['blueTotalExperience'])

    # ゴールドと経験値の相互作用
    df['goldXexperience'] = df['blueTotalGold'] * df['blueTotalExperience']

    # 初期キルの影響
    df['firstBloodKillRatio'] = df['blueFirstBlood'] * df['blueKills']
    
    # キル、デス、アシストの合計
    df['totalCombatPoints'] = df['blueKills'] + df['blueDeaths'] + df['blueAssists']

    # キル・デス・アシスト比率
    total_combat_actions = df['blueKills'] + df['blueDeaths'] + df['blueAssists']
    df['killRatio'] = df['blueKills'] / total_combat_actions
    df['deathRatio'] = df['blueDeaths'] / total_combat_actions
    df['assistRatio'] = df['blueAssists'] / total_combat_actions

    # 効率的なリソース利用
    df['goldPerKill'] = df['blueTotalGold'] / (df['blueKills'] + 1)  # ゼロ除算を防ぐために+1
    df['experiencePerKill'] = df['blueTotalExperience'] / (df['blueKills'] + 1)  # ゼロ除算を防ぐために+1

    # エリートモンスターの討伐効率
    df['eliteMonstersPerGold'] = df['blueEliteMonsters'] / df['blueTotalGold']
    df['eliteMonstersPerExperience'] = df['blueEliteMonsters'] / df['blueTotalExperience']

    # チームの総合力
    df['teamStrength'] = df['blueKills'] + df['blueAssists'] + df['blueEliteMonsters']

    return df

- Apply data

In [6]:
train = create_features(train)
test = create_features(test)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   gameId                      8000 non-null   int64  
 1   blueFirstBlood              8000 non-null   int64  
 2   blueKills                   8000 non-null   int64  
 3   blueDeaths                  8000 non-null   int64  
 4   blueAssists                 8000 non-null   int64  
 5   blueEliteMonsters           8000 non-null   int64  
 6   blueDragons                 8000 non-null   int64  
 7   blueTotalGold               8000 non-null   int64  
 8   blueTotalExperience         8000 non-null   int64  
 9   blueWins                    8000 non-null   int64  
 10  killDeathRatio              8000 non-null   float64
 11  killDeathDiff               8000 non-null   int64  
 12  assistRate                  8000 non-null   float64
 13  blueHerald                  8000 

# 特徴量の効き

In [8]:
# 特徴量間の相関を観察
plt.figure(figsize=(12, 10))
sns.heatmap(train.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

# model

- data

In [ ]:
# データの準備
X = train.drop(['blueWins', 'gameId'], axis=1)
y = train['blueWins']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

test_drop_gameId = test.drop('gameId', axis=1)


# test_drop_gameId_temp = test.drop('gameId', axis=1)

# # 特徴量の標準化
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)  # 検証データも同じスケーラーを使用して標準化

# # テストデータも同じスケーラーを使用して標準化
# test_drop_gameId_scaled = scaler.transform(test_drop_gameId_temp)

# # NumPy配列からPandasデータフレームに変換
# X_train = pd.DataFrame(X_train, columns=X.columns)
# X_val = pd.DataFrame(X_val, columns=X.columns)
# test_drop_gameId = pd.DataFrame(test_drop_gameId_scaled, columns=test_drop_gameId_temp.columns)



- optuna_Lgbm

In [ ]:
def objective(trial):
    # Optunaで調整するハイパーパラメータ
    param = {
        "objective": "binary",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "max_bin": trial.suggest_int("max_bin", 200, 400),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 0.9),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 0.9),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    # クロスバリデーションでのスコア計算
    kf = KFold(n_splits=4, shuffle=True, random_state=71)
    scores = []
    for tr_idx, va_idx in kf.split(X_train):
        tr_x, va_x = X_train.iloc[tr_idx], X_train.iloc[va_idx]
        tr_y, va_y = y_train.iloc[tr_idx], y_train.iloc[va_idx]

        lgb_train = lgb.Dataset(tr_x, tr_y)
        lgb_eval = lgb.Dataset(va_x, va_y, reference=lgb_train)

        callbacks = [lgb.early_stopping(100)]
        model = lgb.train(param, lgb_train, num_boost_round=5000, callbacks=callbacks, valid_sets=lgb_eval)
        preds = model.predict(va_x)
        pred_labels = np.rint(preds)
        score = accuracy_score(va_y, pred_labels)
        scores.append(score)

    return np.mean(scores)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# 最適なパラメータを取得
best_params_lgbm = study.best_params
print("Best params:", best_params_lgbm)

[I 2024-01-22 09:50:31,356] A new study created in memory with name: no-name-2777f29d-1412-4bda-81b1-17d186e7c1d2


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[37]	valid_0's binary_logloss: 0.481658
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[31]	valid_0's binary_logloss: 0.476021
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[33]	valid_0's binary_logloss: 0.493288
Training until validation scores don't improve for 100 rounds


[I 2024-01-22 09:50:32,989] Trial 0 finished with value: 0.7676562499999999 and parameters: {'num_leaves': 231, 'max_bin': 210, 'learning_rate': 0.06877196428728691, 'bagging_fraction': 0.76611988110595, 'bagging_freq': 5, 'feature_fraction': 0.5853091591985744, 'min_child_samples': 15}. Best is trial 0 with value: 0.7676562499999999.


Early stopping, best iteration is:
[37]	valid_0's binary_logloss: 0.480637
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[60]	valid_0's binary_logloss: 0.466098
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[55]	valid_0's binary_logloss: 0.461041
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[38]	valid_0's binary_logloss: 0.482123
Training until validation scores don't improve for 100 rounds


[I 2024-01-22 09:50:33,664] Trial 1 finished with value: 0.77 and parameters: {'num_leaves': 44, 'max_bin': 383, 'learning_rate': 0.0647679634030938, 'bagging_fraction': 0.5736692260807292, 'bagging_freq': 4, 'feature_fraction': 0.8921129506921806, 'min_child_samples': 43}. Best is trial 1 with value: 0.77.


...

Early stopping, best iteration is:
[77]	valid_0's binary_logloss: 0.474093
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[94]	valid_0's binary_logloss: 0.461021
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[205]	valid_0's binary_logloss: 0.45431
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[259]	valid_0's binary_logloss: 0.450597
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[159]	valid_0's binary_logloss: 0.469637
Training until validation scores don't improve for 100 rounds


[I 2024-01-22 09:52:13,615] Trial 98 finished with value: 0.7743749999999999 and parameters: {'num_leaves': 9, 'max_bin': 285, 'learning_rate': 0.07010439735785372, 'bagging_fraction': 0.8930609476773321, 'bagging_freq': 4, 'feature_fraction': 0.5959577747725888, 'min_child_samples': 46}. Best is trial 11 with value: 0.78078125.


Early stopping, best iteration is:
[228]	valid_0's binary_logloss: 0.461528
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.457378
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[68]	valid_0's binary_logloss: 0.45651
Training until validation scores don't improve for 100 rounds


[I 2024-01-22 09:52:14,231] Trial 99 finished with value: 0.776875 and parameters: {'num_leaves': 18, 'max_bin': 293, 'learning_rate': 0.06472938503451008, 'bagging_fraction': 0.875785261365955, 'bagging_freq': 3, 'feature_fraction': 0.4414711499092315, 'min_child_samples': 20}. Best is trial 11 with value: 0.78078125.


Early stopping, best iteration is:
[101]	valid_0's binary_logloss: 0.471817
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[97]	valid_0's binary_logloss: 0.461164
Best params: {'num_leaves': 5, 'max_bin': 268, 'learning_rate': 0.07509725648590564, 'bagging_fraction': 0.5271600717140941, 'bagging_freq': 3, 'feature_fraction': 0.4557026071991971, 'min_child_samples': 30}


- optuna_rf

In [ ]:
def objective_random_forest(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 2, 32),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 150),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 60)
    }

    clf = RandomForestClassifier(**params)
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_random_forest, n_trials=100)
best_params_rf = study_rf.best_params

[I 2024-01-22 09:52:14,247] A new study created in memory with name: no-name-cb9ba646-d2ff-4b98-b396-b0d7f9fa6e17
[I 2024-01-22 09:52:17,545] Trial 0 finished with value: 0.7642185187967954 and parameters: {'n_estimators': 56, 'max_depth': 19, 'min_samples_split': 148, 'min_samples_leaf': 29}. Best is trial 0 with value: 0.7642185187967954.
[I 2024-01-22 09:52:19,993] Trial 1 finished with value: 0.7625004521998736 and parameters: {'n_estimators': 108, 'max_depth': 8, 'min_samples_split': 94, 'min_samples_leaf': 46}. Best is trial 0 with value: 0.7642185187967954.
[I 2024-01-22 09:52:22,695] Trial 2 finished with value: 0.7639064093455324 and parameters: {'n_estimators': 206, 'max_depth': 22, 'min_samples_split': 110, 'min_samples_leaf': 43}. Best is trial 0 with value: 0.7642185187967954.
[I 2024-01-22 09:52:23,859] Trial 3 finished with value: 0.7721866101090947 and parameters: {'n_estimators': 120, 'max_depth': 16, 'min_samples_split': 48, 'min_samples_leaf': 2}. Best is trial 3 wit

- optuna_SVM

In [ ]:
# 1時間でも終わらないので飛ばす


# def objective_svm(trial):
#     params = {
#         "C": trial.suggest_float("C", 0.1, 10.0),
#         "kernel": trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"]),
#         "gamma": trial.suggest_categorical("gamma", ["scale", "auto"])
#     }

#     clf = svm.SVC(**params)
#     return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

# study_svm = optuna.create_study(direction="maximize")
# study_svm.optimize(objective_svm, n_trials=100)
# best_params_svm = study_svm.best_params

- optuna_LR

In [ ]:
def objective_logistic_regression(trial):
    params = {
        "C": trial.suggest_loguniform("C", 1e-4, 10.0),
        "penalty": trial.suggest_categorical("penalty", ["l2", "none"])
    }

    clf = LogisticRegression(**params)
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

study_lr = optuna.create_study(direction="maximize")
study_lr.optimize(objective_logistic_regression, n_trials=100)
best_params_lr = study_lr.best_params

[I 2024-01-22 09:55:09,062] A new study created in memory with name: no-name-bf61db28-9efe-4f9c-aa64-d889f544fcd8
C:\Users\104161\AppData\Local\Temp\ipykernel_31740\3235143177.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "C": trial.suggest_loguniform("C", 1e-4, 10.0),
[I 2024-01-22 09:55:09,215] Trial 0 finished with value: 0.7067197853225954 and parameters: {'C': 0.00031585210322534823, 'penalty': 'l2'}. Best is trial 0 with value: 0.7067197853225954.
C:\Users\104161\AppData\Local\Temp\ipykernel_31740\3235143177.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "C": trial.suggest_loguniform("C", 1e-4, 10.0),
[I 2024-01-22 09:55:09,438] Trial 1 finished with va

- optuna_catboost

In [ ]:
def objective_catboost(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 50, 300),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "random_strength": trial.suggest_int("random_strength", 0, 100),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "od_type": trial.suggest_categorical("od_type", ["IncToDec", "Iter"])
    }

    clf = CatBoostClassifier(**params, verbose=0)
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

study_cb = optuna.create_study(direction="maximize")
study_cb.optimize(objective_catboost, n_trials=100)
best_params_cb = study_cb.best_params


[I 2024-01-22 09:55:26,528] A new study created in memory with name: no-name-a9393553-6567-45bc-86ab-6a12b6368311
[I 2024-01-22 09:55:27,948] Trial 0 finished with value: 0.7485946653157057 and parameters: {'iterations': 74, 'depth': 8, 'learning_rate': 0.04412197468975697, 'random_strength': 35, 'bagging_temperature': 0.4111817718505102, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.7485946653157057.
[I 2024-01-22 09:55:32,113] Trial 1 finished with value: 0.7628110238054124 and parameters: {'iterations': 255, 'depth': 8, 'learning_rate': 0.12444094140349952, 'random_strength': 28, 'bagging_temperature': 0.4463046046920317, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.7628110238054124.
[I 2024-01-22 09:55:34,453] Trial 2 finished with value: 0.7615614143083803 and parameters: {'iterations': 94, 'depth': 8, 'learning_rate': 0.283632169230871, 'random_strength': 67, 'bagging_temperature': 0.9476207177683553, 'od_type': 'Iter'}. Best is trial 1 with value: 0.762811023805

- optuna_XGBoost

In [ ]:
def objective_xgboost(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 2, 25),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0.0, 1.0)
    }

    clf = XGBClassifier(**params)
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(objective_xgboost, n_trials=100)
best_params_xgb = study_xgb.best_params

[I 2024-01-22 09:57:31,255] A new study created in memory with name: no-name-26e8c292-a600-456b-acfa-46dfdd196931
[I 2024-01-22 09:57:32,084] Trial 0 finished with value: 0.7499974735391675 and parameters: {'n_estimators': 202, 'max_depth': 10, 'learning_rate': 0.1459701709609916, 'subsample': 0.7131086962235211, 'colsample_bytree': 0.6770795666130449, 'gamma': 0.5306219092739346}. Best is trial 0 with value: 0.7499974735391675.
[I 2024-01-22 09:57:32,637] Trial 1 finished with value: 0.7651549203813329 and parameters: {'n_estimators': 51, 'max_depth': 14, 'learning_rate': 0.06943212168763094, 'subsample': 0.7089449052570602, 'colsample_bytree': 0.6244709438398365, 'gamma': 0.5631296570605134}. Best is trial 1 with value: 0.7651549203813329.
[I 2024-01-22 09:57:33,720] Trial 2 finished with value: 0.7457796021021913 and parameters: {'n_estimators': 175, 'max_depth': 14, 'learning_rate': 0.23444386404688636, 'subsample': 0.9605019913740982, 'colsample_bytree': 0.6312050309295116, 'gamma

- study

In [ ]:
def predict(X_train, y_train, X_test, mode, X_val):
    preds = []  # バリデーションデータに対する予測結果を格納するリスト
    preds_val = []
    preds_test = []  # テストデータに対する予測結果を格納するリスト

    # クロスバリデーションで予測を実行
    kf = KFold(n_splits=4, shuffle=True, random_state=71)
    for tr_idx, va_idx in kf.split(X_train):
        tr_x, va_x = X_train.iloc[tr_idx], X_train.iloc[va_idx]
        tr_y, va_y = y_train.iloc[tr_idx], y_train.iloc[va_idx]

        # モデルを構築
        if mode == 'LightGBM':
            lgb_train = lgb.Dataset(tr_x, tr_y)
            lgb_eval = lgb.Dataset(va_x, va_y, reference=lgb_train)
            # 早期停止コールバックを設定
            callbacks = [lgb.early_stopping(100)]
            model = lgb.train(best_params_lgbm, lgb_train, num_boost_round=5000, callbacks=callbacks, valid_sets=lgb_eval)

        elif mode == 'RandomForest':
            model = RandomForestClassifier(random_state=123, **best_params_rf)
            model = model.fit(tr_x, tr_y)

        elif mode == 'SVM':
            model = svm.LinearSVC()
            model = model.fit(tr_x, tr_y)

        elif mode == 'LogisticRegression':
            model = LogisticRegression(**best_params_lr)
            model = model.fit(tr_x, tr_y)

        elif mode == 'CatBoost':
            model = CatBoostClassifier(**best_params_cb)
            model = model.fit(tr_x, tr_y)

        elif mode == 'XGBoost':
            model = xgb.XGBClassifier(**best_params_xgb)
            model = model.fit(tr_x, tr_y)

        # バリデーションデータに対する予測
        pred = model.predict(va_x)
        preds.append(pred)

        pred_val = model.predict(X_val)
        preds_val.append(pred_val)

        # テストデータに対する予測
        pred_test = model.predict(X_test)
        preds_test.append(pred_test)

    # バリデーションデータに対する予測結果を連結、元の順序に戻す
    preds = np.concatenate(preds, axis=0)

    # valに対する平均予測値を取得
    preds_val = np.mean(preds_val, axis=0)

    # テストデータに対する平均予測値を取得
    preds_test = np.mean(preds_test, axis=0)

    return preds, preds_val, preds_test

- 学習

In [ ]:
# 異なるモデルのベースモデルを定義
base_models = [
    ('LightGBM', lgb.LGBMClassifier(**best_params_lgbm)),
    ('RandomForest', RandomForestClassifier(random_state=123, **best_params_rf)),
    ('LogisticRegression', LogisticRegression(**best_params_lr)),
    ('XGBoost', XGBClassifier(**best_params_xgb)),
    ('CatBoost', CatBoostClassifier(**best_params_cb))
]

# スタッキングモデルを定義（ここではロジスティック回帰を使用）
stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression(**best_params_lr))
# stacking_model = StackingClassifier(estimators=base_models, final_estimator=RandomForestClassifier(random_state=123))

# ベースモデルのトレーニングと予測
for model_name, model in base_models:
    preds_train, preds_val, preds_test = predict(X_train, y_train, test_drop_gameId, model_name, X_val)  # テストデータを使って予測
    # トレーニングデータの予測結果を特徴として追加
    X_train[model_name] = preds_train
    X_val[model_name] = preds_val
    test_drop_gameId[model_name] = preds_test  # テストデータの特徴量として追加

# スタッキングモデルのトレーニング
stacking_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020976 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1320
[LightGBM] [Info] Number of data points in the train set: 4800, number of used features: 27
[LightGBM] [Info] Start training from score 0.488750
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[360]	valid_0's l2: 0.145844
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000524 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1325
[LightGBM] [Info] Number of data points in the train set: 4800, number of used features: 27
[LightGBM] [Info] Start training from score 0.495000
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[207]	valid_0's l2: 0.145719
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the over

c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\1041

0:	learn: 0.6738463	total: 151ms	remaining: 19.7s
1:	learn: 0.6099214	total: 156ms	remaining: 10.1s
2:	learn: 0.5962587	total: 162ms	remaining: 6.98s
3:	learn: 0.5909301	total: 168ms	remaining: 5.36s
4:	learn: 0.5840787	total: 173ms	remaining: 4.39s
5:	learn: 0.5832792	total: 176ms	remaining: 3.69s
6:	learn: 0.5741701	total: 178ms	remaining: 3.18s
7:	learn: 0.5674413	total: 180ms	remaining: 2.79s
8:	learn: 0.5639081	total: 183ms	remaining: 2.5s
9:	learn: 0.5604177	total: 185ms	remaining: 2.25s
10:	learn: 0.5577600	total: 187ms	remaining: 2.06s
11:	learn: 0.5537464	total: 189ms	remaining: 1.89s
12:	learn: 0.5501172	total: 193ms	remaining: 1.76s
13:	learn: 0.5443430	total: 195ms	remaining: 1.64s
14:	learn: 0.5350893	total: 197ms	remaining: 1.54s
15:	learn: 0.5334115	total: 199ms	remaining: 1.45s
16:	learn: 0.5327726	total: 202ms	remaining: 1.36s
17:	learn: 0.5314304	total: 204ms	remaining: 1.29s
18:	learn: 0.5293464	total: 207ms	remaining: 1.23s
19:	learn: 0.5280776	total: 210ms	remainin

c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0:	learn: 0.6255374	total: 4.93ms	remaining: 646ms
1:	learn: 0.6136636	total: 9.63ms	remaining: 626ms
2:	learn: 0.6028830	total: 14.2ms	remaining: 611ms
3:	learn: 0.5876891	total: 18ms	remaining: 576ms
4:	learn: 0.5838495	total: 21.8ms	remaining: 555ms
5:	learn: 0.5748236	total: 26ms	remaining: 546ms
6:	learn: 0.5732331	total: 31.1ms	remaining: 556ms
7:	learn: 0.5630272	total: 38.7ms	remaining: 600ms
8:	learn: 0.5604036	total: 43.5ms	remaining: 595ms
9:	learn: 0.5577626	total: 51.4ms	remaining: 627ms
10:	learn: 0.5572465	total: 55.5ms	remaining: 611ms
11:	learn: 0.5554058	total: 59ms	remaining: 590ms
12:	learn: 0.5538866	total: 62.6ms	remaining: 573ms
13:	learn: 0.5522515	total: 67.3ms	remaining: 568ms
14:	learn: 0.5507438	total: 71.5ms	remaining: 558ms
15:	learn: 0.5489241	total: 75.3ms	remaining: 546ms
16:	learn: 0.5476915	total: 78.7ms	remaining: 532ms
17:	learn: 0.5475138	total: 82.4ms	remaining: 522ms
18:	learn: 0.5384934	total: 85.6ms	remaining: 509ms
19:	learn: 0.5374190	total: 

c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\1041

0:	learn: 0.6329275	total: 4.82ms	remaining: 631ms
1:	learn: 0.6175052	total: 7.86ms	remaining: 511ms
2:	learn: 0.6013562	total: 10.6ms	remaining: 454ms
3:	learn: 0.5946190	total: 13.8ms	remaining: 440ms
4:	learn: 0.5905239	total: 16.3ms	remaining: 415ms
5:	learn: 0.5796850	total: 19ms	remaining: 399ms
6:	learn: 0.5741096	total: 21.9ms	remaining: 392ms
7:	learn: 0.5607116	total: 25ms	remaining: 387ms
8:	learn: 0.5403588	total: 29.3ms	remaining: 400ms
9:	learn: 0.5383408	total: 32.9ms	remaining: 401ms
10:	learn: 0.5377121	total: 35.6ms	remaining: 392ms
11:	learn: 0.5332517	total: 38.6ms	remaining: 386ms
12:	learn: 0.5302620	total: 41.9ms	remaining: 384ms
13:	learn: 0.5265529	total: 45ms	remaining: 379ms
14:	learn: 0.5252174	total: 47.6ms	remaining: 372ms
15:	learn: 0.5205038	total: 50.7ms	remaining: 368ms
16:	learn: 0.5187363	total: 53.9ms	remaining: 364ms
17:	learn: 0.5179212	total: 57.2ms	remaining: 362ms
18:	learn: 0.5164452	total: 60.2ms	remaining: 358ms
19:	learn: 0.5159944	total: 

c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


StackingClassifier(estimators=[('LightGBM',
                                LGBMClassifier(bagging_fraction=0.5271600717140941,
                                               bagging_freq=3,
                                               feature_fraction=0.4557026071991971,
                                               learning_rate=0.07509725648590564,
                                               max_bin=268,
                                               min_child_samples=30,
                                               num_leaves=5)),
                               ('RandomForest',
                                RandomForestClassifier(max_depth=18,
                                                       min_samples_leaf=8,
                                                       min_samples_split=54,
                                                       n_estimators=285,
                                                       random_state=1...
                                              max_delta_step=None, max_depth=2,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=242, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...)),
                               ('CatBoost',
                                <catboost.core.CatBoostClassifier object at 0x0000022ECF144AF0>)],
                   final_estimator=LogisticRegression(C=2.240068272775415,
                                                      penalty='none'))

- y_val accuracy確認

In [ ]:
# X_valデータでモデルの精度を評価
y_val_pred = stacking_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

[LightGBM] [Warning] feature_fraction is set=0.4557026071991971, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4557026071991971
[LightGBM] [Warning] bagging_fraction is set=0.5271600717140941, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5271600717140941
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
Validation Accuracy: 0.799375


- testにy_test_pred結合

In [ ]:
stacking_test_preds = stacking_model.predict(test_drop_gameId)

[LightGBM] [Warning] feature_fraction is set=0.4557026071991971, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4557026071991971
[LightGBM] [Warning] bagging_fraction is set=0.5271600717140941, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5271600717140941
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


In [ ]:
# 現在の日時を取得してフォーマットする
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output = f'../data/output/{current_time}_submit_cv_stacking_val{val_accuracy:.4f}.csv'

In [ ]:
submit = pd.DataFrame({
    '0': test['gameId'],
    '1': stacking_test_preds
})

# カラムヘッダー消去して出力
# ヘッダーなしでCSVファイルとして保存
submit.to_csv(output, header=False, index=False)

- 出力csvチェック

In [ ]:
check = pd.read_csv(output)

In [ ]:
check

,9,1
0,15,1
1,18,1
2,23,0
3,31,1
4,32,0
...,...,...
1994,9971,0
1995,9980,0
1996,9983,1
1997,9996,1
